In [2]:
import findspark

In [3]:
findspark.init("/usr/local/spark")

In [4]:
from pyspark.sql import SparkSession

In [5]:
# May take awhile locally
spark = SparkSession.builder.appName("Operations2").getOrCreate()

In [7]:
import os
os.getenv('PYTHON_SPARK')

'/home/pasharma/workspace/CodingChills/PySpark'

In [8]:
data_dir = os.path.join(os.getenv('PYTHON_SPARK'),'Data')

In [62]:
df = spark.read.csv(os.path.join(data_dir,'sales_info.csv'),inferSchema=True,header=True)

In [63]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [64]:
df.show(3)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
+-------+-------+-----+
only showing top 3 rows



In [65]:
#lets groupby using company column
df.groupBy("Company")

In [66]:
# Mean
df.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



Check out this link for more info on other methods:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module
Not all methods need a groupby call, instead you can just call the generalized .agg() method, that will call the aggregate across all rows in the dataframe column specified. It can take in arguments as a single column, or create multiple aggregate calls all at once using dictionary notation.
For example:
    

In [67]:
# Max sales across everything
df.agg({'Sales':'max'}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [68]:
grouped = df.groupBy("Company")
grouped.agg({"Sales":'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [69]:
df.groupBy('Company').agg({"Sales":'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [60]:
from pyspark.sql import functions as F

In [71]:
df.groupBy('Company').agg(F.min('Sales'), F.max('Sales')).show()

+-------+----------+----------+
|Company|min(Sales)|max(Sales)|
+-------+----------+----------+
|   APPL|     130.0|     750.0|
|   GOOG|     120.0|     340.0|
|     FB|     350.0|     870.0|
|   MSFT|     124.0|     600.0|
+-------+----------+----------+



## Functions
There are a variety of functions you can import from pyspark.sql.functions. Check out the documentation for the full list available:
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [19]:
from pyspark.sql.functions import countDistinct, avg,stddev

In [20]:
df.select(countDistinct("Sales")).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [21]:
df.select(countDistinct("Sales").alias("Distinct Sales")).show()

+--------------+
|Distinct Sales|
+--------------+
|            11|
+--------------+



In [22]:
df.select(stddev("Sales")).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [23]:
from pyspark.sql.functions import format_number

In [24]:
sales_std = df.select(stddev("Sales").alias('std'))
sales_std.show()

+------------------+
|               std|
+------------------+
|250.08742410799007|
+------------------+



In [25]:
# format_number("col_name",decimal places)
sales_std.select(format_number('std',2)).show()

+---------------------+
|format_number(std, 2)|
+---------------------+
|               250.09|
+---------------------+



In [26]:
# OrderBy
# Ascending
df.orderBy("Sales").show(5)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
+-------+-------+-----+
only showing top 5 rows



In [28]:
# Descending call off the column itself.
df.orderBy(df["Sales"].desc()).show(5)

+-------+------+-----+
|Company|Person|Sales|
+-------+------+-----+
|     FB|  Carl|870.0|
|   APPL|  Mike|750.0|
|   MSFT|  Tina|600.0|
|   APPL| Chris|350.0|
|     FB| Sarah|350.0|
+-------+------+-----+
only showing top 5 rows



## Dates and TimeStamps

In [29]:
df = spark.read.csv(os.path.join(data_dir,"appl_stock.csv"),header=True,inferSchema=True)

In [30]:
df.show(2)

+-------------------+----------+----------+------------------+----------+---------+------------------+
|               Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+-------------------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
+-------------------+----------+----------+------------------+----------+---------+------------------+
only showing top 2 rows



In [31]:
from pyspark.sql.functions import format_number,dayofmonth,hour,dayofyear,month,year,weekofyear,date_format

In [35]:
# day
df.select(dayofmonth(df['Date'])).show(5)

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
+----------------+
only showing top 5 rows



In [36]:
# Month
df.select(month(df['Date'])).show(2)

+-----------+
|month(Date)|
+-----------+
|          1|
|          1|
+-----------+
only showing top 2 rows



In [37]:
df.withColumn("Year",year(df['Date'])).show(2)

+-------------------+----------+----------+------------------+----------+---------+------------------+----+
|               Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|Year|
+-------------------+----------+----------+------------------+----------+---------+------------------+----+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|2010|
|2010-01-05 00:00:00|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|2010|
+-------------------+----------+----------+------------------+----------+---------+------------------+----+
only showing top 2 rows



In [38]:
newdf = df.withColumn("Year",year(df['Date']))
newdf.groupBy("Year").mean()[['avg(Year)','avg(Close)']].show()

+---------+------------------+
|avg(Year)|        avg(Close)|
+---------+------------------+
|   2015.0|120.03999980555547|
|   2013.0| 472.6348802857143|
|   2014.0| 295.4023416507935|
|   2012.0| 576.0497195640002|
|   2016.0|104.60400786904763|
|   2010.0| 259.8424600000002|
|   2011.0|364.00432532142867|
+---------+------------------+



In [39]:
result = newdf.groupBy("Year").mean()[['avg(Year)','avg(Close)']]
result = result.withColumnRenamed("avg(Year)","Year")
result = result.select('Year',format_number('avg(Close)',2).alias("Mean Close")).show()

+------+----------+
|  Year|Mean Close|
+------+----------+
|2015.0|    120.04|
|2013.0|    472.63|
|2014.0|    295.40|
|2012.0|    576.05|
|2016.0|    104.60|
|2010.0|    259.84|
|2011.0|    364.00|
+------+----------+



## Missing Data

Often data sources are incomplete, which means you will have missing data, you have 3 basic options for filling in missing data (you will personally have to make the decision for what is the right approach:

* Just keep the missing data points.
* Drop them missing data points (including the entire row)
* Fill them in with some other value.

Let's cover examples of each of these methods!

In [40]:
df = spark.read.csv(os.path.join(data_dir,"ContainsNull.csv"),header=True,inferSchema=True)

In [41]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [42]:
# Drop any row that contains missing data
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [43]:
# Has to have at least 2 NON-null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [48]:
df.na.drop(subset=["Sales"]).show()
#consider null of only Sales Column

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [47]:
# any row item is null
df.na.drop(how='any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [46]:
# if all row items are null
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Fill the missing values

We can also fill the missing values with new values. If you have multiple nulls across multiple data types, Spark is actually smart enough to match up the data types. For example:

In [49]:
df.na.fill('NEW VALUE').show()
#only string columns will get filled

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|NEW VALUE| null|
|emp3|NEW VALUE|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



In [50]:
df.na.fill(0).show()
#only int columns get filled

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [51]:
df.na.fill('No Name',subset=['Name']).show()
#specify columns as well using subset keyword

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [52]:
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect()


In [55]:
mean_val[0][0]

400.5

In [57]:
mean_sales = mean_val[0][0]

In [58]:
df.na.fill(mean_sales,["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [59]:
# One (very ugly) one-liner
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

